In [1]:
import sys 
import pickle
with open('XGB_ECS111_FinalProject_Group1.pkl', 'rb') as file:
    model = pickle.load(file)

In [3]:
from flask import Flask, render_template_string, request, jsonify
from threading import Thread
import numpy as np

app = Flask(__name__)

html_temp = """
<!DOCTYPE html>
<html>
<head>
<script src="https://code.jquery.com/jquery-3.7.0.js"></script>
<title>tester</title>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">

<style>
    header {
        font-family: 'Georgia';
        font-size: 35px;
        text-align: center;
        text-shadow: 4px 4px 4px rgb(255, 255, 255);
    }
    body {
        background-image: url('static/diab.jpg'); 
        background-size: auto;
        background-repeat: no-repeat;
        background-attachment: fixed;
        font-family: 'Georgia';
        font-size: 16px;
    }
    h1 {
        font-family: 'Georgia';
    }
    .grid-container-element { 
        display: grid; 
        place-items: center;
        grid-template-columns: 1fr 1fr; 
        grid-gap: 30px; 
        width: 90%; 
        padding : 20px
    } 
    .grid-child-element,
    .grid-child-element1 { 
        position: relative;
        background-color: white;
        padding: 10px;
        border: 15px solid #af8b63;
        border-radius: 10px;
        width: 60%; 
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }
    .grid-child-element::before,
    .grid-child-element1::before {
        content: '';
        position: absolute;
        top: -25px; /* Adjust position to overhang */
        left: calc(50% - 100px); /* Adjust position to center horizontally */
        width: 200px; /* Adjust width of the rectangle */
        height: 50px; /* Adjust height of the rectangle */
        background-color: rgb(219, 192, 69); /* Adjust color of the rectangle */
        z-index: 1; /* Ensure it's above the border */
    }
    input {
        background: #fff;
        color: #525865;
        border-radius: 4px;
        border: 1px solid #d1d1d1;
        box-shadow: inset 1px 2px 8px rgba(0, 0, 0, 0.07);
        font-family: inherit;
        font-size: 1em;
        line-height: 1.45;
        outline: none;
        padding: 0.6em 1.45em 0.7em;
        -webkit-transition: .18s ease-out;
        -moz-transition: .18s ease-out;
        -o-transition: .18s ease-out;
        transition: .18s ease-out;
    }
    input:hover {
        box-shadow: inset 1px 2px 8px rgba(0, 0, 0, 0.02);
    }
    input:focus {
        color: #4b515d;
        border: 1px solid #B8B6B6;
        box-shadow: inset 1px 2px 4px rgba(0, 0, 0, 0.01), 0px 0px 8px rgba(0, 0, 0, 0.2);
    }
    input[type="submit"] {
        margin-top: 20px;
        background-color: #007bff;
        color: white;
        padding: 10px 20px;
        border: none;
        border-radius: 5px;
        cursor: pointer;
        font-family: 'Georgia';
        font-size: 15px;
    }
    form input,
    form select,
    form textarea{
        margin-bottom: 20px; 
    }
    .hidden {
        display: none;
    }

</style>

<script type="text/javascript" language="javascript">  
    $(document).ready(function() {  
    });  
</script>

</head>

<body>
    <header> 
        <h1>Diabetes Predictor</h1>
    </header>
    
    <div class="grid-container-element">
        <div id="info" class="grid-child-element">
            <h1>Patient Information:</h1>
            <form id="info-form">
                <label for="age">Age:</label><br>
                <input type="text" id="age" name="age"><br>
                <label for="weight">Weight (lbs):</label><br>
                <input type="text" id="weight" name="weight"><br>
                <label for="height">Height (inches):</label><br>
                <input type="text" id="height" name="height"><br>

                <br><br>

                <label for="sex">Sex:</label>
                <input type="radio" id="sex" name="sex" value=1>
                <label>Male</label>
                <input type="radio" id="sex" name="sex" value=0>
                <label>Female</label><br><br><br>
                
                <label for="bp">Do you have high blood pressure?</label>
                <input type="radio" id="bp" name="bp" value=1>
                <label>Yes</label>
                <input type="radio" id="bp" name="bp" value=0>
                <label>No</label><br><br><br>

                <label for="chol">Do you have high cholesterol?</label>
                <input type="radio" id="chol" name="chol" value=1>
                <label>Yes</label>
                <input type="radio" id="chol" name="chol" value=0>
                <label>No</label><br><br><br>

                <input type="button" id='predict' value="Submit">
            </form>
        </div>
        
        <div id="result" class="grid-child-element1">
            <div id="predicted">
            </div>
        </div>

    </div>

    <script>
        function updateChildSize() {
            // Get the computed style of .grid-child-element
            const clip1 = document.getElementById('info');
            const clipStyle = window.getComputedStyle(clip1);

            // Apply the computed width and height to .grid-child-element1
            const clip2 = document.querySelector('.grid-child-element1');
            clip2.style.width = clipStyle.width;
            clip2.style.height = clipStyle.height;
        }
        updateChildSize();
        window.addEventListener('resize', updateChildSize);
    </script>

    <script> //submit
        $("#predict").on("click", function() {
            age = $("#age").val()
            weight = $("#weight").val();
            height = $("#height").val();
            sex = $("#sex").val();
            bp = $("#bp").val();
            chol = $("#chol").val();
            $.post('/predict', {age:age, weight:weight, height:height, sex:sex, bp:bp, chol:chol}).done(function(response) {
                var diag = parseInt(response);
                if(diag == 1) {
                    $("#predicted").html('<div style="text-align: center;"><br><br><p style="font-size: 35px; font-weight: bold;">You Have Diabetes</p><br><br><img src="static/dy.png" width="200" height="200" onload="$(this).fadeIn()"/></div>');
                } 
                else {
                    $("#predicted").html('<div style="text-align: center;"><br><br><p style="font-size: 35px; font-weight: bold;">You Do Not Have Diabetes</p><br><br><img src="static/dn.png" width="200" height="200" onload="$(this).fadeIn()"/></div>');
                }
                $("#info-form")[0].reset();
            });
        });
    </script>

</body>
</html>
"""

In [4]:
@app.route('/', methods=['GET', 'POST'])
def index():
    return render_template_string(html_temp)

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        age = float(request.form['age'])
        weight = float(request.form['weight'])
        height = float(request.form['height'])
        sex = int(request.form['sex']) # 1 - Male, 0 - Female
        bp = int(request.form['bp']) # 1 - Yes High BP, 0 - No High BP
        chol = int(request.form['chol']) # 1 - Yes High Chol, 0 - No High Chol

        bmi = (weight / (height*height)) * 703 ## calculate BMI
        # Determine AGE Class
        ageclass = getAgeclass(age)

        input_data = np.array([[bmi, bp, chol, sex, ageclass]])
        prediction = model.predict(input_data)
        return str(prediction[0])
        #return jsonify({"prediction": prediction[0]})
       

        #return jsonify({"prediction": age})
def getAgeclass(age):
    ageclass = 1
    if age >=18 and age <= 24:
        return 1
    elif age >=25 and age <= 29:
        return 2
    elif age >=30 and age <= 34:
        return 3
    elif age >=35 and age <= 39:
        return 4
    elif age >=40 and age <= 44:
        return 5
    elif age >=45 and age <= 49:
        return 6
    elif age >=50 and age <= 54:
        return 7
    elif age >=55 and age <= 59:
        return 8
    elif age >=60 and age <= 64:
        return 9
    elif age >=64 and age <= 69:
        return 10
    elif age >=70 and age <= 74:
        return 11
    elif age >=75 and age <= 79:
        return 12
    elif age >= 80:
        return 13
    else:
        return 0
   
       
       

if __name__ == '__main__':

    flask_thread = Thread(target=app.run(port = 8000))
    flask_thread.start()
    flask_thread.join()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [13/Mar/2024 19:41:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 19:41:04] "GET /static/diab.jpg HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2024 19:41:15] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 19:41:15] "GET /static/dn.png HTTP/1.1" 304 -
127.0.0.1 - - [13/Mar/2024 19:41:27] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2024 19:41:27] "GET /static/dy.png HTTP/1.1" 304 -
